In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import math
import numpy as np
import time

from hdbscan import HDBSCAN
from tilesets.models import Tileset

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named 'tilesets'

In [1]:
d = 'gigapan-54825-snapshots'
z = 2
x_from = 0
x_to = 454330
y_from = 0
y_to = 149278
width = 600
height = 200
i_min = 16
i_max = 64
t = 16
n = 5

/Users/Fritz/.jupyter
/Users/Fritz/.jupyter/custom/custom.js
alert("hello world from custom.js")



In [ ]:
def cluster_fragments(
    d,
    z,
    x_from = 0,
    x_to,
    y_from = 0,
    y_to = ,
    width = 1072,
    height = 768,
    i_min = 16,
    i_max = 64,
    t = 16,
    n = 5,
):
    '''Cluster small regions within a larger region
    '''

    if d:
        tile_set_uuids = d.split(',')
    else:
        return JsonResponse({
            'error': 'No tile set uuids specified',
        }, status=400)

    try:
        # Map UUIDs to tile set objects
        tile_sets = list(map(
            lambda uuid: Tileset.objects.get(uuid=uuid),
            tile_set_uuids
        ))
    except Tileset.DoesNotExist as e:
        print(e)
        return JsonResponse({
            'error': 'one or more tile sets were not found',
        }, status=404)

    supported_filetypes = ['bed2ddb', '2dannodb', 'geodb']

    if not all(
        tile_set.filetype in supported_filetypes for tile_set in tile_sets
    ):
        return JsonResponse({
            'error': (
                'One or more tile sets are not supported. Only bed2ddb, '
                '2dannodb, and geodb are supported.'
            ),
        }, status=400)

    # Assemble features
    inset_dims = []
    inset_aspect_ratio = []
    inset_size_min = math.inf
    inset_size_max = -math.inf
    inset_centroids = []

    data_to_view_scale = (x_to - x_from) / width

    feature_area_total = np.zeros([height, width])

    for tile_set in tile_sets:
        features = get_features(
            tile_set, zoom_level, x_from, x_to, y_from, y_to
        )

        for f in features:
            f_width = f[1] - f[0]
            f_height = f[3] - f[2]
            size = max(f_width, f_height) / data_to_view_scale

            d_x_1 = round(f[0] / data_to_view_scale)
            d_x_2 = round(f[1] / data_to_view_scale)
            d_y_1 = round(f[2] / data_to_view_scale)
            d_y_2 = round(f[3] / data_to_view_scale)

            feature_area_total[d_x_1:d_x_2, d_y_1:d_y_2] = 1

            if size <= inset_thres:
                inset_centroids.append(np.mean([f[0:2], f[2:4]], axis=1))
                inset_size_min = min(size, inset_size_min)
                inset_size_max = max(size, inset_size_max)
                inset_dims.append([f_width, f_height])
                inset_aspect_ratio.append(f_width / f_height)

    print('ass', feature_area_total.shape)

    def lin_scl(v, d_from, d_to, r_from, r_to, is_clapped):
        return np.minimum(
            r_to if is_clapped else np.inf,
            np.maximum(
                r_from if is_clapped else -np.inf,
                (((v - d_from) / (d_to - d_from)) * (r_to - r_from)) + r_from
            )
        )

    inset_dims = np.array(inset_dims)

    try:
        inset_dim_max = np.maximum(inset_dims[:, 0], inset_dims[:, 1])
    except Exception as e:
        inset_dim_max = np.array([])

    inset_size_to_disp_scale = lin_scl(
        inset_dim_max,
        inset_size_min,
        inset_size_max,
        inset_disp_size_min,
        inset_disp_size_max,
        True
    ) / inset_dim_max
    inset_disp_size = (inset_dims.T * inset_size_to_disp_scale).T

    # View area
    view_area = width * height
    inset_area_total = np.sum(inset_disp_size)

    num_clust = 0

    if len(inset_centroids) > 0:
        db = HDBSCAN(min_cluster_size=min_cluster_size).fit(inset_centroids)

        # Get labels
        labels = db.labels_

        # "-1" is the cluster of noise
        num_clust = labels.max() + 1

    # Cache results for 30 mins
    rdb.set('clust_frag_%s' % fingerprint, pickle.dumps(results), 60 * 30)

    return {
        'num_clust': int(num_clust),
        'num_insets': len(inset_centroids),
        'view_area': int(view_area),
        'feature_area_total': int(np.sum(feature_area_total)),
        'feature_stress': np.sum(feature_area_total) / view_area,
        'inset_area_total': int(inset_area_total),
        'inset_stress': inset_area_total / view_area,
        'ass': feature_area_total
    }

In [ ]:
cluster_fragments(
    d=d,
    z=z,
    x_from=x_from,
    x_to=x_to,
    y_from=y_from,
    y_to=y_to,
    width=width,
    height=height,
    i_min=i_min,
    i_max=i_max,
    t=t,
    n=n,
)